In [ ]:
from pymongo import MongoClient
import pymongo
db_url= ""
client = MongoClient(db_url)

# See all databases: list(client.list_databases())
airbnb=client['sample_airbnb']['listingsAndReviews']
geospatial=client['sample_geospatial']['shipwrecks']
supplies=client['sample_supplies']['sales']
training=client['sample_training']
weather=client['sample_weatherdata']['data']
grades = training['grades']
movies=client['sample_mflix']["movies"]



db2 = '' 
client2 = MongoClient(db2)
air_routes = client2.aggregations.air_routes
air_airlines = client2.aggregations.air_airlines
parent_reference = client2.aggregations.parent_reference
employees = client2.aggregations.employees
icecream_data = client2.aggregations.icecream_data
stocks = client2.aggregations.stocks
customers = client2.aggregations.customers
air_alliances = client2.aggregations.air_alliances
solarSystem = client2.aggregations.solarSystem
exoplanets = client2.aggregations.exoplanets
nycFacilities = client2.aggregations.nycFacilities

# exoplanets.count_documents({})


## Question 1  

In the grades collection, write a query where:

~~~
1) An object where the "scores.type" is "exam" and "scores.score" is greater than 98.5
2) An object where the "scores.type" is "quiz" and "scores.score" is less than 50
3) An object where the "scores.type" is "homework" and "scores.score" is greater than 97
4) LIMIT to 1 result
5) SORT by "student_id" and class_id descending
~~~

In [ ]:
list(grades
     .find({"$and": [{"scores": {"$elemMatch":
                                 {"type": "exam",
                                  "score": {"$gt": 98.5}}}},
                      {"scores": {"$elemMatch":
                                  {"type": "quiz",
                                   "score": {"$lt": 50}}}},
                     {"scores": {"$elemMatch":
                                 {"type": "homework",
                                  "score": {"$gt": 97}}
                               }}]
            })
     .sort([ ("student_id",-1), ("class_id",-1)])
     .limit(1))

[{'_id': ObjectId('56d5f7f1604eb380b0da5939'),
  'class_id': 216.0,
  'scores': [{'score': 99.9646959940383, 'type': 'exam'},
   {'score': 48.086039950865334, 'type': 'quiz'},
   {'score': 99.98929736978539, 'type': 'homework'},
   {'score': 94.05166065389842, 'type': 'homework'}],
  'student_id': 9841.0}]

## Question 2 

In the supplies collection, write a query where:

~~~
1) AT LEAST ONE element in the array of objects in the "items" field has the following:
    - "quantity" greater than 9
    - "tags" in at least one of the following: "office" or "general"
2) "couponUsed" is False
3) "storeLocation" is in one of the following: "Austin","London" OR where no "items.name" value is equal to "binder"
2) LIMIT to 1 result
3) SORT "saleDate" descending, "storeLocation" ascending
~~~

In [ ]:
list(supplies
     .find({"items": 
            {"$elemMatch": 
             {"quantity": {"$gt":9},
              "tags": {"$in": ["office","general"]}}},
            "couponUsed": False,
            "$or": [
                    {"storeLocation": {"$in": ["Austin", "London"]}},
                    {"items.name": {"$not": {"$eq": "binder"}}}]
     })
     .sort( [("saleDate",-1), ("storeLocation",1)] )
     .limit(1))

[{'_id': ObjectId('5bd761dcae323e45a93cd298'),
  'couponUsed': False,
  'customer': {'age': 55,
   'email': 'zazipu@acisije.is',
   'gender': 'F',
   'satisfaction': 4},
  'items': [{'name': 'binder',
    'price': Decimal128('22.11'),
    'quantity': 10,
    'tags': ['school', 'general', 'organization']}],
  'purchaseMethod': 'Online',
  'saleDate': datetime.datetime(2017, 12, 29, 18, 41, 49, 488000),
  'storeLocation': 'Austin'}]

## Question 3
In the grades collection, write a query where:
~~~
1) IN "scores" array, any/all object(s) with a "type" field value of "homework" must have a "score" field value of greater than 94. In other words not any object with a type of homework can have a score less than 94.
2) LIMIT to 1 result
3) SORT by "student_id" descending, "class_id" descending  
~~~

In [ ]:
list(grades
     .find({"scores":
            {"$not":
             {"$elemMatch":
              {"$and":
               [{"type": "homework"},
                {"score": {"$lt": 94}}]}}
            }})
     .sort( [("student_id",-1), ("class_id",-1)])
     .limit(1))

[{'_id': ObjectId('56d5f7f1604eb380b0da5f40'),
  'class_id': 142.0,
  'scores': [{'score': 87.67256283653538, 'type': 'exam'},
   {'score': 25.344749508969876, 'type': 'quiz'},
   {'score': 95.27036816628505, 'type': 'homework'},
   {'score': 94.97547552861886, 'type': 'homework'}],
  'student_id': 9995.0}]

## Question 4
In the training.zips collection, write a query where:
~~~
1) GROUP by "state", sum the number of documents, and sum the "pop" field
2) SORT "pop" descending 
3) LIMIT to 5 results
~~~

In [ ]:
pipeline = [
            {"$group": {
                "_id": "$state",
                "count": {"$sum": 1},
                "pop": {"$sum": "$pop"}
            }},
            {"$sort": {
                "pop":-1}},
            {"$limit": 5}
]

list(training.zips.aggregate(pipeline))

[{'_id': 'CA', 'count': 1523, 'pop': 29760021},
 {'_id': 'NY', 'count': 1596, 'pop': 17990455},
 {'_id': 'TX', 'count': 1676, 'pop': 16986510},
 {'_id': 'FL', 'count': 826, 'pop': 12937926},
 {'_id': 'PA', 'count': 1458, 'pop': 11881643}]

## Question 5

~~~
SELECT year, COUNT(*) as moviesCount
FROM movies
GROUP BY year
HAVING moviesCount > 1000
ORDER BY moviesCount DESC
~~~

In [ ]:
# remember to GROUP moviesCount! when selecting COUNT(*) moviesCount
pipeline = [
            {"$project": {
                "year":1,
                "moviesCount": {"$sum": 1}}},
            {"$group": {"_id": "$year",
                        "moviesCount": {"$sum": 1}}},
            {"$match": {"moviesCount": {"$gt": 1000}}},
            {"$sort": {"moviesCount": -1}}
]

list(movies.aggregate(pipeline))

[{'_id': 2013, 'moviesCount': 1220},
 {'_id': 2014, 'moviesCount': 1147},
 {'_id': 2012, 'moviesCount': 1109},
 {'_id': 2011, 'moviesCount': 1040}]

## Question 6

In the movies collection, write a query where:

~~~
1) All of the following:  
-"rated" value is "UNRATED" OR "countries" is "USA" OR "num_mflix_comments" is between 10 and 20
- The "awards.wins" is greater than 10
- "genres" has all of the following values: "Comedy","Romance"
5) ONLY RETURN the following values: "rated", "countries","num_mflix_comments","awards.wins","genres", "title"
6) LIMIT to 1
7) Sort "num_mflix_comments" descending and "title" ascending
~~~

In [ ]:
pipeline = [
            {"$match": {
                "$or": [{"rated": "UNRATED"},
                        {"countries": "USA"},
                        {"num_mflix_comments": {"$gt":10, "$lt":20}}],
                "awards.wins": {"$gt": 10},
                "genres": {"$in": ["Comedy", "Romance"]}
            }},
            {"$project": {
                "rated":1, 
                "countries":1,
                "num_mflix_comments":1, 
                "awards.wins":1,
                "genres":1,
                "title":1
            }},
            {"$sort": {"num_mflix_comments":-1, "title":1}},
            {"$limit":1}
]

list(movies.aggregate(pipeline))

[{'_id': ObjectId('573a13a5f29313caabd159a9'),
  'awards': {'wins': 13},
  'countries': ['UK', 'USA', 'France', 'Germany'],
  'genres': ['Comedy', 'Drama', 'Romance'],
  'num_mflix_comments': 441,
  'rated': 'PG-13',
  'title': 'About a Boy'}]

## Question 7

In the training.companies collection, write a query where:

~~~
1) "relationships" array where a single object has a "title" field value of "CFO" and a "person.first_name" field value of "Gary"
2) "relationships" array where the "title" field never has a value of "Board of Directors" and the "title" field exists
3) Return ONLY the following fields: "relationships.title",  "relationships.person.first_name", "name"
4) LIMIT to 1 result
5) SORT "name" in ascending order
~~~

In [ ]:
pipeline = [
            {"$match": {"$and": [
                                 {"relationships": {
                                     "$elemMatch": {"title": "CFO",
                                                    "person.first_name": "Gary"}}},
                                 {"relationships": {
                                     "$elemMatch": {"title": {"$ne": "Board of Directors"},
                                                    "title":{"$exists": True}
                                 }}}]
            }},
            {"$sort": {"name":1}},
            {"$project": {
                "relationships.title":1,
                "relationships.person.first_name":1,
                "name":1
            }},
            {"$limit":1}          
]

list(training.companies.aggregate(pipeline))

[{'_id': ObjectId('52cdef7d4bab8bd675299102'),
  'name': 'KnowNow',
  'relationships': [{'person': {'first_name': 'Todd'},
    'title': 'President and CEO'},
   {'person': {'first_name': 'Bill'}, 'title': 'CEO'},
   {'person': {'first_name': 'Adam'}, 'title': 'Co-Founder and Board'},
   {'person': {'first_name': 'Gary'}, 'title': 'CFO'},
   {'person': {'first_name': 'Sam'},
    'title': 'Vice President of Technical Services'},
   {'person': {'first_name': 'Satish'}, 'title': 'VP Engineering'},
   {'person': {'first_name': 'Rohit'}, 'title': 'co-founder'},
   {'person': {'first_name': 'Amanda'},
    'title': 'Business Development and Sales Intern'},
   {'person': {'first_name': 'Issac'}, 'title': 'Product Manager'},
   {'person': {'first_name': 'Niloy'}, 'title': ''},
   {'person': {'first_name': 'Tommy'}, 'title': 'Client Architect'},
   {'person': {'first_name': 'Ryan'},
    'title': 'Vice President of Marketing'}]}]

## Question 8

In the training.companies collection, write a query where:

~~~
1) "funding_rounds" array of objects has one element/item/object where
    -"round_code" is not in the following values: "a","b","unattributed","seed",'angel' and exists
    -"funded_year" is greater than 2005 and less than or equal to 2008
    - "raised_amount" is greater than 5000000 OR "raised_currency_code" is not equal to "USD"
2) LIMIT to 1 result
3) SORT by "founded_year" descending, "name" ascending
4) Return the following fields "funding_rounds.round_code","funding_rounds.funded_year","funding_rounds.raised_amount",
         "funding_rounds.investments","funding_rounds.raised_currency_code","founded_year", "name"
~~~

In [ ]:
pipeline = [
            {"$match": {
                "funding_rounds": {
                    "$elemMatch": {"round_code": {"$not": {"$in":
                                                   ["a", "b", "unattributed", "seed", "angel"]}, "$exists": True},
                                    "funded_year": {"$gt": 2005, "$lte": 2008},
                                    "$or": [
                                           {"raised_amount": {"$gt": 5000000}},
                                           {"raised_currency_code": {"$ne": "USD"}}]}}
            }},
            {"$sort": {"founded_year": -1,"name": 1
            }},
            {"$limit":1},
            {"$project": {
                "funding_rounds.round_code": 1,
                "funding_rounds.funded_year":1,
                "funding_rounds.raised_amount":1,
                "funding_rounds.investments":1,
                "funding_rounds.raised_currency_code":1,
                "founded_year":1,
                "name":1}}
]

list(training.companies.aggregate(pipeline))

[{'_id': ObjectId('52cdef7e4bab8bd67529ab2f'),
  'founded_year': 2009,
  'funding_rounds': [{'funded_year': 2008,
    'investments': [{'company': None,
      'financial_org': {'name': 'Sequoia Capital',
       'permalink': 'sequoia-capital'},
      'person': None},
     {'company': None,
      'financial_org': {'name': 'Lightspeed Venture Partners',
       'permalink': 'lightspeed-venture-partners'},
      'person': None},
     {'company': None,
      'financial_org': {'name': 'TL Ventures', 'permalink': 'tl-ventures'},
      'person': None},
     {'company': None,
      'financial_org': {'name': 'ABS Ventures', 'permalink': 'abs-ventures'},
      'person': None}],
    'raised_amount': 8000000,
    'raised_currency_code': 'USD',
    'round_code': 'd'}],
  'name': 'ScaleMP'}]

## Question 9

In the supplies collection:

~~~
1) Select/project the couponUsed, and create the follow fields:
-"avg_quantity" = average of the "items.quantity" field 
-"max_price" = maximum of the "items.price" field
2) Group by the couponUsed field, and compute the following fields:
- "avg_quantity" = average of the "avg_quantity" field
-"max_price" = maximum of the "max_price" price field
-"count" =  the number of documents in the grouping
3) Order by the "max_price" field descending
~~~

In [ ]:
pipeline = [
            {"$project": {
                "couponUsed": 1,
                "avg_quantity": {"$avg": "$items.quantity"},
                "max_price": {"$max": "$items.price"}}},
            {"$group": {
                "_id": "$couponUsed",
                "avg_quantity": {"$avg": "$avg_quantity"},
                "max_price": {"$max": "$max_price"},
                "count": {"$sum": 1}
            }},
            {"$sort": {
                "max_price": -1
            }}
]

list(supplies.aggregate(pipeline))

[{'_id': False,
  'avg_quantity': 4.03899952272161,
  'count': 4523,
  'max_price': Decimal128('1599.49')},
 {'_id': True,
  'avg_quantity': 4.017110911450534,
  'count': 477,
  'max_price': Decimal128('1595.35')}]

## Question 10

In the grades collection:

~~~
1) The "scores" array has no object with "type" field value of "homework" and "score" less than 90. (e.g. {type:"homework",score:80} should not pass this filter)
2) project the "class_id", and set the "avg_score" field equal to the average of the "scores.score" fields
3) group on class_id, set the "class_avg" to the average of the "avg_score" field, and set the "count" to the number of documents in the grouping
4) Order by count descending, _id ascending
5) limit to 3 results
~~~

In [ ]:
pipeline = [
            {"$match": 
             {"scores": {
                "$not": {"$elemMatch": {"type": "homework",
                                        "score": {"$lt": 90}}}}
            }},
            {"$project": {
                "class_id":1,
                "avg_score": {"$avg": "$scores.score"}}},
            {"$group": {
                "_id": "$class_id",
                "class_avg": {"$avg": "$avg_score"},
                "count": {"$sum": 1}
            }},
            {"$sort": {
                "count": -1,
                "_id": 1
            }},
            {"$limit": 3}
]

list(grades.aggregate(pipeline))

[{'_id': 142.0, 'class_avg': 74.49470669501589, 'count': 7},
 {'_id': 352.0, 'class_avg': 76.52568914087034, 'count': 7},
 {'_id': 15.0, 'class_avg': 64.64920120380651, 'count': 6}]